In [1]:
# Homework 01
# Hunter Rogers 
# Machine Learning

In [20]:
import pandas as pd
import numpy as np
import math

pd.set_option('display.max_rows', None)

In [19]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
display(train_df)

display(train_df['label'].unique())

len(train_df['label'].unique())

1. In this problem, we will work with a modified version of the badges data we saw
class. We are given names of the ML researchers along with features, and the task is
to assign a positive or a negative label to their badge. There are four features:


(a) is the length of the first name > 5 (lenFirst): Yes or No

(b) is the length of the last name > 5 (lenLast): Yes or No

(c) is the first letter of the first and last name same (sameFirst): Yes or No

(d) the most frequent vowel that is not ‘u’ (vowel): a, e, i, o

(a) [5 points] How many possible functions are there to map these four features to
a boolean decision? How many functions are consistent with the given training
dataset?

There are 1024 possible functions and only one that fits the data (since the function is created from the data exactly)

In [6]:
options = 2 * 2 * 2 * 4

functions = options * options

functions

1024

(b) [3 points] What is the entropy of the labels in this data? Remember that when
calculating entropy, the base of the logarithm should be 2.

In [ ]:
Name lenFirst lenLast sameFirst vowel Badge
Pieter Bartlett Yes Yes No e +ve
George Berg Yes No No e −ve
Hiroshi Motoda Yes Yes No o +ve
Filippo Neri Yes No No i −ve
Robert Roos Yes No Yes o +ve
Satinder Singh Yes No Yes i +ve
Maja Mataric No Yes Yes a +ve
Arun Sharma No Yes No a −ve
Michael Meystel Yes Yes Yes e −ve

In [21]:
# initialise data of lists.
data = {'Name':['Pieter Bartlett', 'George Berg', 'Hiroshi Motoda', 'Filippo Neri', 'Robert Roos', 'Satinder Singh', 'Maja Mataric', 'Arun Sharma', 'Michael Meystel'],
 'lenFirst':['Yes','Yes','Yes','Yes','Yes','Yes','No','No','Yes'],
 'lenLast':['Yes','No','Yes','No','No','No','Yes','Yes','Yes'],
 'sameFirst':['No','No','No','No','Yes','Yes','Yes','No','Yes'],
 'vowel':['e','e','o','i','o','i','a','a','e'],
 'Badge':['+','-','+','-','+','+','+','-','-']}
 
# Create DataFrame
df = pd.DataFrame(data)
df = df.set_index('Name')

In [ ]:
print(df['Badge'].unique()[0])

In [32]:
df.columns.drop(['lenFirst'])

Index(['lenLast', 'sameFirst', 'vowel', 'Badge'], dtype='object')

In [17]:
# def calc_entropy(df, predicted_column):
#     if len(df[predicted_column].unique() == 2):
#         correct = df[predicted_column].value_counts()[0]
#         incorrect = len(df) - correct
#         overall_total = len(df)
#         if(incorrect == 0):
#             overall_entropy = 0
#         else:
#             overall_entropy = -(correct/overall_total) * math.log((correct/overall_total),2) - (incorrect/overall_total) * math.log((incorrect/overall_total),2)
#         for column in df.columns:
#             temp_avg_entropy = 0
#             for value in df[column].unique():
#                 temp_df = df[df[column] == value]
#                 #print(f"{column}: {temp_df[predicted_column].value_counts()[0]}")
                
#                 temp_correct = temp_df[predicted_column].value_counts()[0]
#                 temp_incorrect = len(temp_df) - temp_correct
#                 total = len(temp_df)
#                 if(temp_incorrect == 0):
#                     entropy = 0
#                 else:
#                     entropy = -(temp_correct/total) * math.log((temp_correct/total),2) - (temp_incorrect/total) * math.log((temp_incorrect/total),2)
#                 #print(f"For {value} in {column}: $$H(Badge) = -({temp_correct}/{total})log_2({temp_correct}/{total}) - ({temp_incorrect}/{total})log_2({temp_incorrect}/{total}) = {round(entropy, 2)}$$")
#                 temp_avg_entropy += (entropy * total)
#             temp_avg_entropy = temp_avg_entropy / len(df)
#             info_gain = overall_entropy - temp_avg_entropy
#             print(f'{info_gain} {overall_entropy} {temp_avg_entropy}')
            
#     else:
#         print('nah')



In [33]:
def calc_entropy(df, predicted_column):
    overall_entropy = 0
    for label in df[predicted_column].unique():
        correct = len(df[df[predicted_column] == label])
        overall_total = len(df)
        overall_entropy -= (correct/overall_total) * math.log((correct/overall_total),2)

    for column in df.columns.drop([predicted_column]):
        temp_avg_entropy = 0
        for value in df[column].unique():
            temp_df = df[df[column] == value]
            #print(f"{column}: {temp_df[predicted_column].value_counts()[0]}")
            temp_entropy = 0
            for label in temp_df[predicted_column].unique():
                temp_correct = len(temp_df[temp_df[predicted_column] == label])
                temp_total = len(temp_df)
                temp_entropy -= (temp_correct/temp_total) * math.log((temp_correct/temp_total),2)
            #print(f"For {value} in {column}: $$H(Badge) = -({temp_correct}/{total})log_2({temp_correct}/{total}) - ({temp_incorrect}/{total})log_2({temp_incorrect}/{total}) = {round(entropy, 2)}$$")
            temp_avg_entropy += (temp_entropy * temp_total)
        temp_avg_entropy = temp_avg_entropy / len(df)

        info_gain = overall_entropy - temp_avg_entropy
        print(f'{info_gain} {overall_entropy} {temp_avg_entropy}')
            

In [25]:
def calc_collision_entropy(df, predicted_column):
    overall_entropy = 0
    for label in df[predicted_column].unique():
        correct = len(df[df[predicted_column] == label])
        overall_total = len(df)
        overall_entropy += (correct/overall_total) * (correct/overall_total)
    overall_entropy = -math.log(overall_entropy,2)
    for column in df.columns:
        temp_avg_entropy = 0
        for value in df[column].unique():
            temp_df = df[df[column] == value]
            #print(f"{column}: {temp_df[predicted_column].value_counts()[0]}")
            temp_entropy = 0
            for label in temp_df[predicted_column].unique():
                temp_correct = len(temp_df[temp_df[predicted_column] == label])
                temp_total = len(temp_df)
                temp_entropy += (temp_correct/temp_total) * (temp_correct/temp_total)
            temp_entropy = -math.log(temp_entropy, 2)
            #print(f"For {value} in {column}: $$H(Badge) = -({temp_correct}/{total})log_2({temp_correct}/{total}) - ({temp_incorrect}/{total})log_2({temp_incorrect}/{total}) = {round(entropy, 2)}$$")
            temp_avg_entropy += (temp_entropy * temp_total)
        temp_avg_entropy = temp_avg_entropy / len(df)

        info_gain = overall_entropy - temp_avg_entropy
        print(f'{info_gain} {overall_entropy} {temp_avg_entropy}')
            

In [12]:
temp_correct = 5
temp_incorrect = 4
total = 9

entropy = -(temp_correct/total) * math.log((temp_correct/total),2) - (temp_incorrect/total) * math.log((temp_incorrect/total),2)

entropy

0.9910760598382222

In [36]:
calc_entropy(df, 'Badge')

0.0025652873671375698 0.9910760598382222 0.9885107724710847
0.007214618474517542 0.9910760598382222 0.9838614413637047
0.09109100760379174 0.9910760598382222 0.8999850522344305
0.24053300404228128 0.9910760598382222 0.7505430557959409
vowel


In [26]:
calc_collision_entropy(df, 'Badge')

0.9822979982665411 0.9822979982665411 0.0
0.0049673782239426645 0.9822979982665411 0.9773306200425984
0.013733291803411873 0.9822979982665411 0.9685647064631292
0.15681244508668413 0.9822979982665411 0.825485553179857
0.25518791830378007 0.9822979982665411 0.727110079962761
0.9822979982665411 0.9822979982665411 0.0


In [23]:
df['Badge'].mode()

0    +
Name: Badge, dtype: object

In [3]:
def buildID3(df, predicted_column, max_depth=5, path=pd.Series(), current_depth=0, tree=pd.DataFrame()):
    #Calculate the overall entropy of the dataframe
    

    overall_entropy = 0
    for label in df[predicted_column].unique():
        correct = len(df[df[predicted_column] == label])
        overall_total = len(df)
        overall_entropy -= (correct/overall_total) * math.log((correct/overall_total),2)
    if overall_entropy == 0:
        path.at[current_depth] = label
        #print(path)
        temp = pd.DataFrame(path).T
        #display(temp)
        tree = pd.concat([tree, temp], axis=0, ignore_index=True)
    
    elif (current_depth / 2) == max_depth:
        path.at[current_depth] = df[predicted_column].mode()[0]
        temp = pd.DataFrame(path).T
        tree = pd.concat([tree, temp], axis=0, ignore_index=True)

    else:
    #Find the next node
        highest_gain = 0
        for column in df.columns.drop([predicted_column]):
            temp_avg_entropy = 0
            for value in df[column].unique():
                temp_df = df[df[column] == value]
                #print(f"{column}: {temp_df[predicted_column].value_counts()[0]}")
                temp_entropy = 0
                for label in temp_df[predicted_column].unique():
                    temp_correct = len(temp_df[temp_df[predicted_column] == label])
                    temp_total = len(temp_df)
                    temp_entropy -= (temp_correct/temp_total) * math.log((temp_correct/temp_total),2)
                #print(f"For {value} in {column}: $$H(Badge) = -({temp_correct}/{total})log_2({temp_correct}/{total}) - ({temp_incorrect}/{total})log_2({temp_incorrect}/{total}) = {round(entropy, 2)}$$")
                temp_avg_entropy += (temp_entropy * temp_total)
            temp_avg_entropy = temp_avg_entropy / len(df)

            info_gain = overall_entropy - temp_avg_entropy
            if info_gain > highest_gain:
                highest_gain = info_gain
                split_on = column
            #print(f'{info_gain} {overall_entropy} {temp_avg_entropy}')
        path.at[current_depth] = split_on
        current_depth += 1

        for value in np.append(df[split_on].unique(), 'UNKNOWN'):
            if value == 'UNKNOWN':
                path.at[current_depth] = value
                current_depth += 1
                end_this_branch = int(current_depth/2)
                tree = buildID3(df, predicted_column, end_this_branch, path, current_depth, tree)
                path.at[current_depth] = ''
                current_depth -= 1
                path.at[current_depth] = ''
            else:
                path.at[current_depth] = value
                current_depth += 1
                tree = buildID3(df[df[split_on] == value], predicted_column, max_depth, path, current_depth, tree)
                path.at[current_depth] = ''
                current_depth -= 1
                path.at[current_depth] = ''
    return tree


C:\Users\Hunter Rogers\AppData\Local\Temp\ipykernel_16248\3566628735.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def buildID3(df, predicted_column, max_depth=5, path=pd.Series(), current_depth=0, tree=pd.DataFrame()):


In [28]:
def evaluate_model(tree, test_df, predicted_column):
    counter = 0
    test_df['prediction'] = None
    #print(len(test_df))
    for i in range (0, len(tree)):
        #temp_df = test_df[test_df[] == tree and test_df[] == tree and test_df[] == tree and test_df[] == tree and test_df[] == tree and test_df[] == tree]
        temp_df = test_df
        for row_num in range(0,int((len(tree.columns)+1)/2)):
            #print(row_num)
            if tree.iloc[i, row_num*2] not in test_df.columns:
                #print(tree.iloc[i, row_num*2])
                for index in temp_df.index:
                    test_df['prediction'].loc[index] = tree.iloc[i, row_num*2]
                break
            else:
                if len(temp_df[temp_df[tree.iloc[i, row_num*2]] == tree.iloc[i, row_num*2+1]]) > 0:
                    temp_df = temp_df[temp_df[tree.iloc[i, row_num*2]] == tree.iloc[i, row_num*2+1]]
                else:
                    temp_df = temp_df[temp_df[tree.iloc[i, row_num*2]] == 'UNKNOWN']
        counter += len(temp_df)
    
    test_df['prediction'].fillna(test_df[predicted_column].mode())

    return test_df
    
        
            

In [29]:
tree = buildID3(train_df, 'label', 200)

In [30]:
predictions = evaluate_model(tree, test_df, 'label')

In [ ]:
display(tree)

In [ ]:
#display(predictions.head(1000))



print(np.append(predictions['prediction'].unique(), 'hi'))

print(predictions['prediction'].value_counts())

print(len(predictions))

In [31]:
display(predictions)

,buying,maint,doors,persons,lug_boot,safety,label,prediction
0,high,high,4,more,small,med,unacc,unacc
1,high,low,5more,4,small,high,acc,acc
2,vhigh,vhigh,2,4,med,high,unacc,unacc
3,vhigh,med,5more,more,med,med,acc,None
4,high,high,3,more,small,high,acc,acc
5,med,med,4,4,big,high,vgood,vgood
6,low,med,5more,more,small,high,good,None
7,high,high,3,2,med,low,unacc,unacc
8,high,low,4,2,med,high,unacc,unacc
9,low,vhigh,3,4,med,low,unacc,unacc


In [ ]:
print